In [3]:
# biblioteca que tem o joguinho do taxi
import gym
import random

random.seed(1234)

# criando o ambiente do taxi
streets = gym.make("Taxi-v3", render_mode='ansi').env

# resetando o ambiente
streets.reset()
# mostra o estado atual do jogo
print('\n' + streets.render())


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [6]:
# definindo um estado inicial específico 
# linha do taxi, coluna do taxi, passageiro, destino
initial_state = streets.encode(2, 3, 2, 0)

streets.s = initial_state # coloca o ambiente atual no estado inicial definido

streets.reset() # reseta o ambiente
print('\n' + streets.render()) # mostra o estado atual do jogo


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [8]:
# mostra a tabela Q com: probabilidade de cada ação, próximo estado, pontuação e estado do jogp
streets.P[initial_state]

{0: [(1.0, 368, -1, False)],
 1: [(1.0, 168, -1, False)],
 2: [(1.0, 288, -1, False)],
 3: [(1.0, 248, -1, False)],
 4: [(1.0, 268, -10, False)],
 5: [(1.0, 268, -10, False)]}

In [ ]:
import numpy as np

# cria uma tabela Q zerada, com o número de estados e ações possíveis
q_table = np.zeros([streets.observation_space.n, streets.action_space.n])

learning_rate = 0.1 # taxa de aprendizado, evita que o taxi mude de opinão facilmente
discount_factor = 0.6 # valorização das recompensas
exploration = 0.1 # a chance de tomar uma decisão aleatória inves de uma já conhecida
epochs = 10000 # número de partidas para treinar

for taxi_run in range(epochs):

    state = streets.reset()[0] # reinicia o jogo e pega o estado inicial
    done = False

    while not done: # enquanto não tiver acabado
        
        random_value = random.uniform(0, 1) # gera uma probabilidade aleatória

        if (random_value < exploration): # decide se vai explorar ou manter um caminho conhecido
            action = streets.action_space.sample() # a função sample() escolhe uma ação aleatória

        else:
            action = np.argmax(q_table[state]) # olha a linha da tabela q da posição em que ele está e escolhe a ação com maior valor

        # move o taxi
        # next_state é o lugar onde o taxi parou
        # reward é a pontuação que ele recebeu
        # terminated ou truncated indicam se o jogo acabou
        next_state, reward, terminated, truncated, info = streets.step(action) 

        # verifica se o jogo acabou
        done = terminated or truncated
        
        # pega o valor antigo da tabela
        prev_q = q_table[state, action]

        # atualiza a tabela Q usando a formula de qlearning
        next_max_q = np.max(q_table[next_state])
        new_q = (1 - learning_rate) * prev_q + learning_rate * (reward + discount_factor * next_max_q)
        q_table[state, action] = new_q

        # atualiza a posição do taxi
        state = next_state


In [12]:
# mostra as notas da tabela para o estado inicial
q_table[initial_state]

array([-2.42792015, -2.42592733, -2.41888932, -2.3639511 , -8.39843486,
       -6.87487463])

In [15]:
# biblioteca visual do jupyter
from IPython.display import clear_output
from time import sleep

# numero da viagem
for tripnum in range(1, 11):

    # pega o estado inicial
    state = streets.reset()[0]

    done = False

    # enquanto nao tiver acabado
    while not done:

        # a ação é a que tem maior valor na tabela Q
        action = np.argmax(q_table[state])

        # faz a ação
        next_state, reward, terminated, truncated, info = streets.step(action)
        done = terminated or truncated

        # espera um pouco e limpa a tela para mostrar o próximo estado
        clear_output(wait=True)

        # mostra o número da viagem e o estado atual do jogo
        print("Trip number" + str(tripnum))

        # mostra o jogo
        print(streets.render())

        # espera
        sleep(.5)

        # passa para o próximo estado
        state = next_state

    sleep(2)

Trip number10
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)



### Your Challenge

Modify the block above to keep track of the total time steps, and use that as a metric as to how good our Q-learning system is. You might want to increase the number of simulated trips, and remove the sleep() calls to allow you to run over more samples

Now, try experimenting with the hyperparameters. How low can the number of epochs go before our model starts to suffer? Can you come up with better learning rates, discount factors, or exploration factors to make the training more efficient? The exploration vs exploitation rate in particular is interesting to experiment with.

In [25]:
def test(trips):

    steps = 0

    for _ in range(trips):

        # pega o estado inicial
        state = streets.reset()[0]
        done = False

        # enquanto nao tiver acabado
        while not done:

            # a ação é a que tem maior valor na tabela Q
            action = np.argmax(q_table[state])

            # faz a ação
            next_state, reward, terminated, truncated, info = streets.step(action)
            done = terminated or truncated

            # atualiza o número de passos
            steps += 1

            # passa para o próximo estado
            state = next_state

    avg_step = steps / trips
    print("Average steps per trip: {}".format(avg_step))
    return avg_step

Chegou a um ponto em que eu tive que interromper o taxi, o motivo é simples: ele estava perdido, o treinamento não foi o suficiente para que ele conseguisse realizar todos os caminhos, ele provavelmente está se perdendo no meio e eu não coloquei nenhum limite de tempo ou algo do tipo para interrupção

No próximo bloco vou fazer algumas mudanças no código de treinamento que está lá encima.
Colocarei apenas os comentários nas alterações feitas, e excluirei os comentários antigos para evitar poluição visual


In [24]:
import numpy as np

q_table = np.zeros([streets.observation_space.n, streets.action_space.n])

learning_rate = 0.075 # diminui a taxa de aprendizado para ver se ele se torna mais estável
discount_factor = 0.8 # aumentei a taxa de desconto, para que ele valorize as recompensas futuras 
exploration = 0.25 # aumentei os indices de descobrimento para que ele explore mais o ambiente
epochs = 30000 # tripliquei o número de treinamentos 

for taxi_run in range(epochs):

    state = streets.reset()[0] 
    done = False

    while not done: 
        
        random_value = random.uniform(0, 1) 

        if (random_value < exploration): 
            action = streets.action_space.sample() 

        else:
            action = np.argmax(q_table[state]) 

        next_state, reward, terminated, truncated, info = streets.step(action) 

        done = terminated or truncated
        prev_q = q_table[state, action]
        next_max_q = np.max(q_table[next_state])
        new_q = (1 - learning_rate) * prev_q + learning_rate * (reward + discount_factor * next_max_q)
        q_table[state, action] = new_q

        state = next_state


In [26]:
test(1000)

Average steps per trip: 13.006


13.006